# 1. Packages and Data

## 1.1 Packages

In [115]:
# Standard data libraries
import pandas as pd
import numpy as np

# Other useful ones
import os
import time

#For displaying tables nicer (side by side etc)
from IPython.display import display_html 

## 1.2 Data

This is the wine dataset from UCI ml repository again. Various chemical properties of wine. I have made an addition in that the second last column is named `quality_binary` and is the judge score (1-10) which I have truncated in the last column to a binary 1-0 (good-bad) if the `quality_raw` is greater than or equal to 7.

In [4]:
data_dir = os.getcwd() + "/datasets"
wine = pd.read_csv(data_dir + "/wine-total.csv")

# 2. Pandas part 2

Here we continue from last week and do a bit more _wrangling_ with pandas. There are many tools under the hood of Pandas to assist with your data manipulation and analysis. I will outline some useful functions and resources, however feel free to continue to work through the docs to find and build examples for yourself.

## 2.1 Key topics

You can see below there are lots of useful methods with pandas. To assist prioritisation of your time, the following are the most important to understand at this point in time. Though all may be useful for your task!

* Selecting & Dropping
* Functions on a df

Groupby is likely to also be useful for your EDA

* Selecting data from a df:
    * Base Methods
        * Extract a column or columns to list, df or series
        * Using the '.' notation vs using square brackets (df.column vs df['column'] vs df[['column']])
        * How to extract a column to a numpy array
        * Slicing out rows
    * Loc and Iloc
    * Ensure you know how to extract multiple columns (by name, list or index)
    * Extraction via conditions (including multiple conditions, OR conditions)
* Dropping rows, columns, conditional dropping
* Reshape a df
    * Transpose
    * Melt (turning wide to long)
    * Pivot & Pivot_table (long to wide)
    * Crosstab analysis
    * Binning (cut and qcut)
* Joining dfs 
    * Merge, concat
* Sorting a df
    * Using python's `sorted` method (if you extract a series)
    * Directly on a df (`sort_values`). 
        * Note that when you sort_values() on a df this will still keep the index values. Hence to 'put it back' you can sort_index which will sort by the index. If we do not want this to be possible anymore, we can reset the index with reset_index() first. 
* Functions on a df
    * apply() vs applymap() vs map()
    * using .agg function on columns
    * vectorising to improve speed
    * pipe functions
* Grouping a df
    * simple groupby
        * Undertake one and see what kind of object is returned
    * Including a call to .agg
    * Multi-index groupby

## 2.2 Resources

### 2.2.1 New Resources

Largely the resources from last week will still apply, so I will copy them below. Some additionals resources are below that are more focussed on the content.

* https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

These cheat sheets. However they don't cover the nuances of actually doing this stuff, combining techniques etc. Hence you are encouraged to build your own notes and examples in addition to these.

* https://nikgrozev.com/2015/07/01/reshaping-in-pandas-pivot-pivot-table-stack-and-unstack-explained-with-pictures/

This tutorial on stacking, pivoting etc a df. I don't see stack and unstack used as much but these are a generalisation of pivot so perhaps may be useful. 

* http://pbpython.com/pandas-crosstab.html

Nice tutorial on crosstab analysis

* http://jonathansoma.com/lede/foundations/classes/pandas%20columns%20and%20functions/apply-a-function-to-every-row-in-a-pandas-dataframe/

Functions and dataframes

### 2.2.2 Resources from last week

Copied across for convenience:

* https://www.datacamp.com/courses/intermediate-python-for-data-science
    * Chapter 2 and the back end of chapter 3
* https://jakevdp.github.io/PythonDataScienceHandbook/
    * Chapter 3

I would strongly recommend simply going through the pandas documentation and making notes and examples on all the useful elements. This does take a bit of time and you will not be able to do it all at once, but at least getting yourself familiar with what is in there will be very useful.

* https://pandas.pydata.org/pandas-docs/stable/reference/series.html
    * The pandas series reference documentation
* https://pandas.pydata.org/pandas-docs/stable/reference/frame.html
    * The pandas dataframe reference documentation

# 3. Some of my notes

I realise the list above is quite large and so I have included some of my notes and walkthroughs to assist. This more advanced stuff is not compulsory for undertaking deep learning. Basic python is and being *moderately* comfortable with pandas and its abilitities is where I want to get you to. However I put this content in here for those who want to work ahead and perhaps others who want to come back to it later. Enjoy!

I won't provide my notes on selecting as this is such an important topic you need to write your own notes and practice this. Some of the other more complex or lower value (so you can just flick through) ones I will outline below.

## 3.1 Binning values (cut + qcut)

These two functions are used to bin continuous values. The documentation notes 'For example, cut could convert ages to groups of age ranges. Supports binning into an equal number of bins, or a pre-specified array of bins.'

### 3.1.1 Cut

For this function we will explore examples where we:
* Ask to split into n bins
* define the bins ourselves (useful for dealing with a long tail)

The 'labels' parameter is important to specify. We can give labels to each of the new categories (for example, 'good, bad, ok') or we can just set to False and the bin number for each entity will be returned.

In [73]:
#Let us sample from the wine set
seed = 42
wine_small = wine.sample(50, random_state=seed)

# Have a look to see what types of raw quality we got in the sam[;e]
wine_small.quality_raw.value_counts().sort_index()

4     2
5    19
6    16
7    11
8     2
Name: quality_raw, dtype: int64

In [78]:
# We can start with simply asking to put into 3 distinct bins
bins = pd.cut(wine_small.quality_raw, bins=3, labels=["bad", "ok", "great"], retbins=True)

# Note we are setting retbins to true so we can see what the calculated bins are.
# This means that 'bins' is a tuple, the first element is the bins array, the second is the bin limits
print(bins[0].head(5)) # Print the bin of the first 5 entries
print()
print("The bin limits were: \n")
print(bins[1])
wine_small["quality_cut"] = bins[0] #We can assign the bins back to the dataframe
print()
print(wine_small.iloc[:,12:15].head(20)) # See our column. Just taking last columns and head of 20.

# We can also check the value counts as well
print()
print(wine_small.quality_cut.value_counts().sort_index()) #Compare this to above shows us what was put into what category

3103      bad
1419    great
4761    great
4690    great
4032       ok
Name: quality_raw, dtype: category
Categories (3, object): [bad < ok < great]

The bin limits were: 

[0.993      3.33333333 5.66666667 8.        ]

      quality_raw  quality_binary quality_cut
3103            1               1         bad
1419            7               1       great
4761            6               0       great
4690            6               0       great
4032            5               0          ok
1297            7               1       great
1773            6               0       great
5584            5               0          ok
561             5               0          ok
5946            6               0       great
1891            5               0          ok
2264            6               0       great
6485            6               0       great
217             5               0          ok
230             4               0          ok
2168            7               1       great

In [79]:
#Now let us set the bins explicitly
bins2 = pd.cut(wine_small.quality_raw, bins=[3,5,7, np.inf], labels=["bad", "ok", "great"])
# Note we do not need retbins since we set the bins. 

# Also note that the default ('right' parameter) is to include the rightmost value in that bin
bins2.head(5)
print()
wine_small["quality_cut_2"] = bins2 #We can assign the bins back to the dataframe
print()
print(wine_small.iloc[:,12:].head(10))

# We can also check the value counts as well
print()
print(wine_small.quality_cut_2.value_counts().sort_index()) #Compare this to above shows us what was put into what category



      quality_raw  quality_binary quality_cut quality_cut_2
3103            1               1         bad           NaN
1419            7               1       great            ok
4761            6               0       great            ok
4690            6               0       great            ok
4032            5               0          ok           bad
1297            7               1       great            ok
1773            6               0       great            ok
5584            5               0          ok           bad
561             5               0          ok           bad
5946            6               0       great            ok

bad      21
ok       26
great     2
Name: quality_cut_2, dtype: int64


Try setting an extreme value here and how that affects the bins. Run the cell below then go above, rerun all the cells in this section (AFTER the sampling)

In [77]:
## Set an extreme value to see the effects on the bins
wine_small.iloc[0,-4] = 1

We notice some important insights from using the extreme value:
* When we set our own bins above, this simply drags down the bin limits, but all are included
* When we set our own bins and have a first bin value higher than the extreme there is a NaN value

### 3.1.2 QCut

Whilst cut sets bin widths to be even with respect to the **values of the column**, qcut sets bin widths to have an **equal number of rows in each bin**.

In [80]:
# We can ask for 3 qcuts and see how this affects things
bins_qcut = pd.qcut(wine_small.quality_raw, q=3, labels=["bad", "ok", "great"], retbins=True)
print(bins_qcut[0].head(5))
print("\n The bin edges were: \n")
print(bins_qcut[1])
wine_small["bins_qcut"] = bins_qcut[0]
print(wine_small.iloc[:,12:].head(10))

# Now the value counts should be more interesting
print()
print(wine_small.bins_qcut.value_counts().sort_index())
# Note that they are not quite even, this is because when values are equal they are given the same label

3103      bad
1419    great
4761       ok
4690       ok
4032      bad
Name: quality_raw, dtype: category
Categories (3, object): [bad < ok < great]

 The bin edges were: 

[1. 5. 6. 8.]
      quality_raw  quality_binary quality_cut quality_cut_2 bins_qcut
3103            1               1         bad           NaN       bad
1419            7               1       great            ok     great
4761            6               0       great            ok        ok
4690            6               0       great            ok        ok
4032            5               0          ok           bad       bad
1297            7               1       great            ok     great
1773            6               0       great            ok        ok
5584            5               0          ok           bad       bad
561             5               0          ok           bad       bad
5946            6               0       great            ok        ok

bad      22
ok       16
great    12
Name: b

In [81]:
# However with enough random, different numbers, this works better
np.random.seed(42)

y = pd.Series(np.random.randint(low=1, high=100000, size=500))
quartiles = pd.qcut(y, 4, labels=['1st', '2nd', '3rd', '4th'], retbins=True)

print("Bin limits: \n")
print(quartiles[1])
print()
print('Quartiles:')
print(quartiles[0].value_counts(sort=False))

Bin limits: 

[  207.   23867.75 52254.5  75530.   99714.  ]

Quartiles:
1st    125
2nd    125
3rd    125
4th    125
dtype: int64


## 3.2 Joining Dfs (Merge, concat)

Two useful functions are those related to joining dfs together. Concat and merge have some distinct differences:

* Concat is just putting on df on top (or next to) another to merge. 
    * You need to therefore be sure that columns/rows line up (**ORDERED**)
* Merge is a SQL-like operation that allows for using a key

There is also 'append' for simply adding more rows, though merge and concat can be used for this as well

Additionally there is a 'join' option which performs a merge that defaults to joining on indices. 

### 3.2.1 Concat

In [83]:
# Here is a simple example
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})

df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                     'B': ['B4', 'B5', 'B6', 'B7']})

dfs = [df1,df2]
result1 = pd.concat(dfs) #Notice how we need to feed in a list, not individual dfs ('iterable')

# We can also reset the index using 'ignore_index'
result2 = pd.concat(dfs, ignore_index=True)



In [84]:
print(result1)
print(result2)

    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3
0  A4  B4
1  A5  B5
2  A6  B6
3  A7  B7
    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4
5  A5  B5
6  A6  B6
7  A7  B7


In [85]:
# We could also set labels for our individual parts to make them easy to pull out later
result3 = pd.concat(dfs, keys=("FIRST", "SECOND"))
result3_second = result3.loc["SECOND"]

In [86]:
df1_styler = result3.style.set_table_attributes("style='display:inline;margin:2vw'").set_caption('Adding Keys')
df2_styler = result3_second.style.set_table_attributes("style='display:inline'").set_caption('Original second df')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

Adding Keys 
 
 
 
 A 
 B 
 
 
 FIRST 
 0 
 A0 
 B0 
 
 1 
 A1 
 B1 
 
 2 
 A2 
 B2 
 
 3 
 A3 
 B3 
 
 SECOND 
 0 
 A4 
 B4 
 
 1 
 A5 
 B5 
 
 2 
 A6 
 B6 
 
 3 
 A7 
 B7 
 
 
 Original second df 
 
 
 A 
 B 
 
 
 0 
 A4 
 B4 
 
 1 
 A5 
 B5 
 
 2 
 A6 
 B6 
 
 3 
 A7 
 B7

In [87]:
# Append predates concat and could also be used with less options
df_app = df1.append(df2)
df_app

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3
0,A4,B4
1,A5,B5
2,A6,B6
3,A7,B7


In [88]:
# We can concat different numbers of dimensions
s1 = pd.Series(['X0', 'X1', 'X2'], name='X')

df_dims = pd.concat([df1, s1], axis=1)

df_dims #Notice the NaN where no value present

,A,B,X
0,A0,B0,X0
1,A1,B1,X1
2,A2,B2,X2
3,A3,B3,NaN


### 3.2.2 Merge

This has the options of left, right, outer, inner which align with SQL joins of LEFT OUTER, RIGHT OUTER, FULL OUTER and INNER joins. 

This example used is from https://chrisalbon.com/python/data_wrangling/pandas_join_merge_dataframe/

Another good one seems to be https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html

There is also a useful argument **suffixes** for adding a suffix to columns when there are overlapping column names.

In [89]:
#First we make some dfs of exam marks
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5', '4', '5', '6', '7', '8'],
        'first_name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung','Billy', 'Brian', 'Bran', 'Bryce', 'Betty'], 
        'last_name': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches','Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']}
df_a = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5', '7', '8', '9', '10', '11'],
        'test_id': [51, 15, 15, 61, 16, 14, 15, 1, 61, 16]}
df_b = pd.DataFrame(raw_data, columns = ['subject_id','test_id'])

In [90]:
df1_styler = df_a.style.set_table_attributes("style='display:inline;margin:2vw'").set_caption('Student Details')
df2_styler = df_b.style.set_table_attributes("style='display:inline;margin:2vw'").set_caption('Exam Marks')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches
5,4,Billy,Bonder
6,5,Brian,Black
7,6,Bran,Balwner
8,7,Bryce,Brice
9,8,Betty,Btisan


In [91]:
student_marks_df = df_a.merge(df_b, how='left', on='subject_id')
student_marks_df

,subject_id,first_name,last_name,test_id
0,1,Alex,Anderson,51.0
1,2,Amy,Ackerman,15.0
2,3,Allen,Ali,15.0
3,4,Alice,Aoni,61.0
4,5,Ayoung,Atiches,16.0
5,4,Billy,Bonder,61.0
6,5,Brian,Black,16.0
7,6,Bran,Balwner,NaN
8,7,Bryce,Brice,14.0
9,8,Betty,Btisan,15.0


## 3.3 Sorting the df

In [92]:
# Here is a little toy dataset on cars
names = ['United States', 'Australia', 'Japan', 'India', 'Russia', 'Morocco', 'Egypt']
dr =  [True, False, False, False, True, True, True]
cpc = [809, 731, 588, 18, 200, 70, 45]
cars = pd.DataFrame({ 'country':names, 'drives_right':dr, 'cars_per_cap':cpc })
print(cars)

         country  drives_right  cars_per_cap
0  United States          True           809
1      Australia         False           731
2          Japan         False           588
3          India         False            18
4         Russia          True           200
5        Morocco          True            70
6          Egypt          True            45


### 3.3.1 Sorting an extracted series

You can apply pythons base sorted() method directly to an extracted series object

In [93]:
sorted(cars["cars_per_cap"])

[18, 45, 70, 200, 588, 731, 809]

### 5.2 Sorting on the df

Note that when you sort_values() on a df this will still keep the index values. Hence to 'put it back' you can sort_index which will sort by the index.

If we do not want this to be possible anymore, we can reset the index with reset_index()

In [94]:
cars = cars.reset_index()
cars

,index,country,drives_right,cars_per_cap
0,0,United States,True,809
1,1,Australia,False,731
2,2,Japan,False,588
3,3,India,False,18
4,4,Russia,True,200
5,5,Morocco,True,70
6,6,Egypt,True,45


In [95]:
#Note we could also sort on the 'country' and it would sort alphabetically
cars_sorted = cars.sort_values(by="cars_per_cap", axis=0)
cars_sorted

,index,country,drives_right,cars_per_cap
3,3,India,False,18
6,6,Egypt,True,45
5,5,Morocco,True,70
4,4,Russia,True,200
2,2,Japan,False,588
1,1,Australia,False,731
0,0,United States,True,809


In [96]:
#We could do this in reverse using the "ascending" argument which is defaulted to True
cars_sorted_rev = cars.sort_values(by="cars_per_cap", axis=0, ascending=False)
cars_sorted_rev

,index,country,drives_right,cars_per_cap
0,0,United States,True,809
1,1,Australia,False,731
2,2,Japan,False,588
4,4,Russia,True,200
5,5,Morocco,True,70
6,6,Egypt,True,45
3,3,India,False,18


In [97]:
#sort_values can take a list of columns so we can sort by a number in order
cars_sorted_multiple = cars.sort_values(by=["cars_per_cap", "country"], axis=0)
cars_sorted_multiple

,index,country,drives_right,cars_per_cap
3,3,India,False,18
6,6,Egypt,True,45
5,5,Morocco,True,70
4,4,Russia,True,200
2,2,Japan,False,588
1,1,Australia,False,731
0,0,United States,True,809


## 3.4 Applying functions to dfs

This is a very powerful way to efficiently apply functions to entire rows or columns. The result can then be saved to a new column or kept separate for analysis.

Built-in functions can be used such as np.sum or np.mean or any other defined function.

Some examples taken from here http://jonathansoma.com/lede/foundations/classes/pandas%20columns%20and%20functions/apply-a-function-to-every-row-in-a-pandas-dataframe/

### 3.4.1 apply() & applymap

#### 3.4.1.1 apply()

In [139]:
#Create a simple dataframe
names = ["John", "Steve", "Abby"]
amounts = ["$1.00", "$544.34", "$10.22"]
columns = ["names", "amounts"]
df = pd.DataFrame(list(zip(names, amounts)), columns=columns) 
df

,names,amounts
0,John,$1.00
1,Steve,$544.34
2,Abby,$10.22


In [140]:
#Let us start with a custom function to format some dollar strings
def money_to_float(money_str):
    return float(money_str.replace("$","").replace(",",""))

df['amounts'].apply(money_to_float) #Simply calling apply will return a series

0      1.00
1    544.34
2     10.22
Name: amounts, dtype: float64

In [142]:
# So we could therefore turn that series into a new column
df["amount_clean"] = df["amounts"].apply(money_to_float)
df

,names,amounts,amount_clean
0,John,$1.00,1.00
1,Steve,$544.34,544.34
2,Abby,$10.22,10.22


In [144]:
# Create a dataframe from a list of dictionaries
rectangles = [
    { 'height': 40, 'width': 10 },
    { 'height': 20, 'width': 9 },
    { 'height': 3.4, 'width': 4 }
]

rectangles_df = pd.DataFrame(rectangles)
rectangles_df

,height,width
0,40.0,10
1,20.0,9
2,3.4,4


In [145]:
# We can also send through each row when passing axis=1
# This passes each row through as a series which can be indexed on
rectangles_df_simple = rectangles_df.copy()

def calculate_area(row):
    return row['height'] * row['width']

rectangles_df_simple["area"] = rectangles_df_simple.apply(calculate_area, axis=1)
rectangles_df_simple

,height,width,area
0,40.0,10,400.0
1,20.0,9,180.0
2,3.4,4,13.6


#### 3.4.1.2 applymap()

Where apply can be undertaken on a single row or column, we may want an operation undertaken on every element in the dataframe, in which we use applymap().

There is also a map() method which works on a series object

In [146]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'), index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0.699092,0.599897,-0.481248
Ohio,-2.093794,-0.420047,0.139204
Texas,0.001814,0.755935,1.933543
Oregon,0.413875,0.567768,0.148182


In [147]:
frame = frame.applymap(lambda x: round(x,2))
frame
# Note how it returns a df so we need to assign back to frame

,b,d,e
Utah,0.70,0.60,-0.48
Ohio,-2.09,-0.42,0.14
Texas,0.00,0.76,1.93
Oregon,0.41,0.57,0.15


### 3.4.2 Lambda functions

A lambda function is a small anonymous function. A lambda function can take any number of arguments, but can only have one expression.

In [148]:
#A simple example would be:
x = lambda a, b, c : a + b + c
print(x(5, 6, 2))

13


In [154]:
x

<function __main__.<lambda>(a, b, c)>

In [156]:
# However we can use this to apply anonymous functions to our df if they are simple enough
rectangles_df_lamb = rectangles_df.copy()
rectangles_df_lamb["area"] = rectangles_df.apply(lambda x: x["height"] * x["width"], axis=1)
rectangles_df_lamb

,height,width,area
0,40.0,10,400.0
1,20.0,9,180.0
2,3.4,4,13.6


### 3.4.3 Aggregations

Unlike apply, the aggregate function (use 'agg') is more flexible, allowing multiple, fast aggregations to be done on different columns. 

In [157]:
#Let us use the iris dataset for this
from sklearn.datasets import load_iris

iris = load_iris()
iris = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])
iris.shape

(150, 5)

In [158]:
print(iris.agg({'sepal width (cm)': 'min', 'petal width (cm)': 'max'})) #This returns a series
print(iris.agg({'sepal width (cm)': ['min', 'median'], 'sepal length (cm)': ['min', 'mean']})) #Returns a df

sepal width (cm)    2.0
petal width (cm)    2.5
dtype: float64
        sepal width (cm)  sepal length (cm)
mean                 NaN           5.843333
median               3.0                NaN
min                  2.0           4.300000


### 3.4.4 Vectorisation in pandas

Whilst most built-in python and numpy functions will work in a vectorised format, custom functions using apply loop over the entire dataset. Hence vectorising this approach would be more optimal.

In [159]:
#Let us consider an example where we (for no real reason) add the pH to the alcohol in our wine dataset
wine_apply = wine.copy()

In [160]:
def ph_and_alc(row):
    return row["pH"] + row["alcohol"]

In [161]:
start_time = time.time()
wine_apply["pH_alc"] = wine_apply.apply(ph_and_alc, axis=1)
total_time_apply = time.time() - start_time
print("--- {} seconds ---".format(total_time_apply))

--- 0.15908217430114746 seconds ---


In [162]:
#Make a new copy for our vectorised test
wine_vect = wine.copy()

In [163]:
#Now let us try a vectorised approach
start_time = time.time()
wine_vect["pH_alc"] = ph_and_alc(wine_vect)
total_time_vect = time.time() - start_time
print("--- {} seconds ---".format(total_time_vect)) 
# Note the huge speedup

--- 0.0011568069458007812 seconds ---


In [164]:
#We could even go one step further and use a numpy ndarray if the index does not matter
#This would involve passing in the .values of a series object

## 3.5 Groupby

Groupby is so important it has its own section for now.

Aggregations become very powerful when used in conjunction with groupby. Additionally this is a very important way of being able to group categorical data (and then apply operations on them).

This is where we notice the real flexibility of groupby.agg vs groupby.apply since the agg allows multiple functions meaning we can do this once to a single grouped-by object

### 3.5.1 Simple Groupby

In [165]:
#Start with a simple dataset
g_df = pd.DataFrame(
    {'col1':['A','A','A','A','A','B','B','B','C','C'],
    'col2':[1,2,3,4,5,6,7,8,9,0],
     'col3':[-1,-2,-3,-4,-5,-6,-7,-8,-9,0]
    }
)
g_df

,col1,col2,col3
0,A,1,-1
1,A,2,-2
2,A,3,-3
3,A,4,-4
4,A,5,-5
5,B,6,-6
6,B,7,-7
7,B,8,-8
8,C,9,-9
9,C,0,0


In [166]:
#Firstly we can do a single level groupby
#We will then aggregate, getting the max in col2 FOR the col1 group (and same for min)
simple_g = g_df.groupby('col1').agg({'col2': 'max', 'col3': 'min'})
print(simple_g)
print("shape: ", simple_g.shape)
print(simple_g.index)

      col2  col3
col1            
A        5    -5
B        8    -8
C        9    -9
shape:  (3, 2)
Index(['A', 'B', 'C'], dtype='object', name='col1')


In [167]:
simple_g.index
#Therefore we could 'loc' to get one of the groups back if we wanted

Index(['A', 'B', 'C'], dtype='object', name='col1')

In [168]:
#We can remove the index by either reseting the index or passing in as_index=False
simple_g = simple_g.reset_index()
simple_g #See how the index is now just numbers

,col1,col2,col3
0,A,5,-5
1,B,8,-8
2,C,9,-9


### 3.5.2 Multi-index Groupby

In [169]:
multi_g = g_df.groupby('col1').agg({'col2': ['max', 'min', 'std'], 
                                 'col3': ['size', 'std', 'mean', 'max']})
multi_g

col2               col3                   
      max min       std size       std mean max
col1                                           
A       5   1  1.581139    5  1.581139 -3.0  -1
B       8   6  1.000000    3  1.000000 -7.0  -6
C       9   0  6.363961    2  6.363961 -4.5   0

In [170]:
#Now we could do some selecting such as

#All of column 2
print(multi_g["col2"])
print("")

#Max of column 2
print(multi_g["col2"]["max"])

      max  min       std
col1                    
A       5    1  1.581139
B       8    6  1.000000
C       9    0  6.363961

col1
A    5
B    8
C    9
Name: max, dtype: int64


In [171]:
#We could also flatten the levels if we wanted
multi_g.columns = ["_".join(col) for col in multi_g.columns]
multi_g

,col2_max,col2_min,col2_std,col3_size,col3_std,col3_mean,col3_max
col1,,,,,,,
A,5,1,1.581139,5,1.581139,-3.0,-1
B,8,6,1.000000,3,1.000000,-7.0,-6
C,9,0,6.363961,2,6.363961,-4.5,0


In [172]:
#We could even do a deeper groupby with two levels for the initial column
g_df["col4"] = ['X','X','Y','Y','Y','Y','Y','Y','X','X']
g_df

,col1,col2,col3,col4
0,A,1,-1,X
1,A,2,-2,X
2,A,3,-3,Y
3,A,4,-4,Y
4,A,5,-5,Y
5,B,6,-6,Y
6,B,7,-7,Y
7,B,8,-8,Y
8,C,9,-9,X
9,C,0,0,X


In [173]:
#Here we see grouping by two top columns
multi_g2 = g_df.groupby(['col1', 'col4']).agg('max')
multi_g2

col2  col3
col1 col4            
A    X        2    -1
     Y        5    -3
B    Y        8    -6
C    X        9     0

### 3.5.3 Using groupby for a new variable

Say we want to use the average 'Col2' for each group and have this as a new variable as 'av_col2_by_col1'. We need to take our result and turn it back into a series object so it can be easily used as a variable.

The transform method returns an object that is indexed the same (same size) as the one being grouped. 

In [174]:
g_df

,col1,col2,col3,col4
0,A,1,-1,X
1,A,2,-2,X
2,A,3,-3,Y
3,A,4,-4,Y
4,A,5,-5,Y
5,B,6,-6,Y
6,B,7,-7,Y
7,B,8,-8,Y
8,C,9,-9,X
9,C,0,0,X


In [175]:
g_df['av_col2_by_col1'] = g_df.groupby('col1')["col2"].transform('mean')
g_df

,col1,col2,col3,col4,av_col2_by_col1
0,A,1,-1,X,3.0
1,A,2,-2,X,3.0
2,A,3,-3,Y,3.0
3,A,4,-4,Y,3.0
4,A,5,-5,Y,3.0
5,B,6,-6,Y,7.0
6,B,7,-7,Y,7.0
7,B,8,-8,Y,7.0
8,C,9,-9,X,4.5
9,C,0,0,X,4.5


# 4. Introduction to linear algebra

Linear algebra is a huge topic in and of itself. Do not worry about going too deeply into this, but you will need to know (at the very least) matrix multiplication and derivitives.

<h3 align="center"> Why Math?</h3>
<img src="https://s3-ap-southeast-2.amazonaws.com/mdsi-deep-learn-aut-19/linear_alg_ironman.jpg" width="450" height="450"/>
<style>
 img {
    vertical-align: middle;
}
</style>

## 4.1 Key topics

* Matrices Intro
    * What is a matrix (compared to scalar, vector)
    * Matrix (matrix-vector) multiplication
* Matrix derivatives
    * How to differentiate a matrix? What is the 'Jacobian'

## 4.2 Resources

Some resources:

* https://explained.ai/matrix-calculus/index.html
    * This is extensive and you can skim large chunks, however you can read:
        * From 'Review: Scalar derivative rules' (the above we will cover in class) to the end of 'Generalization of the Jacobian'. Then you are allowed to skim ;)
        * Though of course I recommend doing as much as you can!
        * Here is the arXiv entry if you want a pdf https://arxiv.org/abs/1802.01528v2
* https://www.deeplearningbook.org/contents/linear_algebra.html
    * A concise overview of relevant linear algebra terms and concepts. From 2.7 onwards is not as vital but before that it is important.
* http://d2l.ai/chapter_crashcourse/linear-algebra.html
    * A more practical overview with code examples. 
* https://the-learning-machine.com/article/machine-learning/linear-algebra
    * This resource is quite extensive and there is is a lot in there not as super relevant. This is highly visual and well explained. I would recommend working through the terminology of the first/second resources and looking it up in this resource to see if there are visuals or an alternate explanation to cement the concept.
